In [1]:
%load_ext autoreload
%autoreload 2 

In [2]:
import datetime
from dateutil.relativedelta import relativedelta
import json
import logging
from pyspark.sql import DataFrame, Window, functions as f
from pyspark.sql import SQLContext
from pyspark.sql.types import LongType
import yaml

from common.libs import dates as dates_lib
from common.libs import features_discovery
from common.libs.features_executor import FeaturesExecutor
from common.libs.feature_engineering import max_look_back_monthly_features, max_look_back_daily_weekly_features
from common.libs.zscore import enrich_with_z_score
from common.factory.wrangling_execution_strategy import get_wrangling_execution_strategy
from common.factory.eval_flow_definition import get_evaluation_flow_definition
from common.factory.domain_definition import get_domain_definition
from common.notebook_utils.wrangling.wrangling_execution_strategy import WranglingExecutionStrategy
from common.definitions.domain import DomainDefinition
from common.definitions.eval_flow import EvaluationFlowDefinition
from common.libs.context_utils import get_dataset

from thetaray.api.context import init_context
from thetaray.api.dataset import dataset_functions
from thetaray.api.solution import IngestionMode
from thetaray.common import Constants
from thetaray.common.data_environment import DataEnvironment

logging.getLogger().handlers[0].setFormatter(logging.Formatter(fmt='%(levelname)s: %(asctime)s @ %(message)s',datefmt='%Y-%m-%d %H:%M:%S'))
logging.basicConfig(level=logging.INFO)

import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


from thetaray.api.context import init_context
import datetime
from thetaray.common import Constants

from common.libs.config.loader import load_config
from common.libs.config.basic_execution_config_loader import BasicExecutionConfig, DevBasicExecutionConfig
from common.libs.context_utils import is_run_triggered_from_airflow



with open('/thetaray/git/solutions/domains/demo_fuib/config/spark_config.yaml') as spark_config_file:
    spark_config = yaml.load(spark_config_file, yaml.FullLoader)['spark_config_a']

execution_date=datetime.datetime(1970, 1, 1)

context = init_context(domain='demo_fuib',
                       execution_date=execution_date,
                       spark_conf=spark_config,
                       spark_master='local[*]',
                       allow_type_changes=True)

spark = context.get_spark_session()
sc = SQLContext(spark)
params = context.parameters
print(f"Spark UI URL: {context.get_spark_ui_url()}")

print(json.dumps(params, indent=4))

/thetaray/venv/lib64/python3.11/site-packages/starlette/config.py:60: UserWarning: Config file '.env' not found.
  warnings.warn(f"Config file '{env_file}' not found.")
/thetaray/venv/lib64/python3.11/site-packages/starlette/config.py:60: UserWarning: Config file '.env' not found.
  warnings.warn(f"Config file '{env_file}' not found.")
2025-09-21 15:43:48,978:INFO:thetaray.common.logging:start loading solution.....[ load_risks=True , solution_path=/thetaray/git/solutions/domains , settings_path=/thetaray/git/solutions/settings ]
2025-09-21 15:43:48,988:WARNING:thetaray.common.logging:Encryption is enabled on dataset demo_ret_smb_customer_insights, but data encryption is disabled in deployment
2025-09-21 15:43:49,002:WARNING:thetaray.common.logging:Encryption is enabled on dataset demo_ret_indiv_customer_insights, but data encryption is disabled in deployment
2025-09-21 15:43:49,136:WARNING:thetaray.common.logging:Encryption is enabled on dataset demo_digital_wallets_customer_insights, 

Added `alias` successfully.


INFO: 2025-09-21 15:44:26 @ === Finished updating schema for Evaluation Flows on Minio ===


Added `alias` successfully.
Spark UI URL: https://jupyterhub-platform-thetalab.sonar.thetaray.cloud/user/andre.gutnik/proxy/4040/jobs/
{}


/opt/spark/python/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [3]:
# import psycopg2
# import os
# # Define connection to DB
# username = "postgres"
# password = "postgres"

# shared_namespace = os.environ['SHARED_NAMESPACE']
# conn = psycopg2.connect(
#     host=f"postgres.{shared_namespace}.svc.cluster.local",
#     dbname="cdd",
#     user=username,
#     password=password,
#     port=5432
# )

# query = f"""SELECT * FROM solution_sonar.trx_enriched;
#         """
# df = pd.read_sql_query(query, conn)
# df.shape

# df = df.drop(columns=['tr_job_ts', 'tr_id', 'tr_timestamp', 'tr_partition'])

In [50]:
from pyspark.sql.types import (
    StructType, StructField,
    StringType, BooleanType, DoubleType, LongType, TimestampType
)

from pyspark.sql.types import (
    StructType, StructField,
    StringType, BooleanType, DoubleType, TimestampType
)

# schema_loose = StructType([
#     StructField("txn_id", StringType(), True),
#     StructField("txn_ts", TimestampType(), True),
#     StructField("customer_id", StringType(), True),
#     StructField("account_id", StringType(), True),
#     StructField("customer_type", StringType(), True),
#     StructField("customer_name", StringType(), True),
#     StructField("date_of_birth", TimestampType(), True),
#     StructField("address_full", StringType(), True),
#     StructField("occupation", StringType(), True),
#     StructField("pep_indicator", BooleanType(), True),
#     StructField("industry_code", StringType(), True),

#     # CHANGED: LongType -> DoubleType
#     StructField("account_open_months", DoubleType(), True),

#     StructField("whitelist_flag", BooleanType(), True),
#     StructField("direction", StringType(), True),
#     StructField("method", StringType(), True),
#     StructField("amount", DoubleType(), True),
#     StructField("currency", StringType(), True),
#     StructField("amount_usd", DoubleType(), True),
#     StructField("purpose_code", StringType(), True),
#     StructField("cross_border_flag", BooleanType(), True),
#     StructField("declared_counterparty_name", StringType(), True),
#     StructField("balance_after_txn", DoubleType(), True),
#     StructField("is_split_component", BooleanType(), True),
#     StructField("cp_id", StringType(), True),
#     StructField("cp_type", StringType(), True),
#     StructField("cp_jurisdiction", StringType(), True),
#     StructField("cp_known_shell_flag", BooleanType(), True),
#     StructField("cp_is_bank_customer", BooleanType(), True),
#     StructField("counterparty_name", StringType(), True),
#     StructField("counterparty_address", StringType(), True),
#     StructField("counterparty_ip", StringType(), True),
#     StructField("endpoint_type", StringType(), True),
#     StructField("ip_hash", StringType(), True),
#     StructField("device_id_hash", StringType(), True),
#     StructField("geo_lat", DoubleType(), True),
#     StructField("geo_lon", DoubleType(), True),
#     StructField("atm_id", StringType(), True),
#     StructField("withdrawal_channel", StringType(), True),
#     StructField("pos_terminal_id", StringType(), True),
#     StructField("rep_id_hash", StringType(), True),
#     StructField("address_hash", StringType(), True),
#     StructField("website_hash", StringType(), True),
#     StructField("scenario_type", StringType(), True),
#     StructField("scenario_role", StringType(), True),
#     StructField("anomaly_flag", BooleanType(), True),
#     StructField("cp_country_risk_level", StringType(), True),
#     StructField("is_new_account", BooleanType(), True),

#     # CHANGED: LongType -> DoubleType
#     StructField("day_offset", DoubleType(), True),
#     StructField("month_offset", DoubleType(), True),

#     StructField("year_month", TimestampType(), True),
#     StructField("is_me_to_me", BooleanType(), True),
#     StructField("dataset_type", StringType(), True),
# ])



spark_df = spark.createDataFrame(df)

spark_df.count()

600806

In [51]:
spark_df = spark_df.withColumn("account_open_months", spark_df["account_open_months"].cast("long"))

In [53]:
from thetaray.common.data_environment import DataEnvironment
dataset_functions.write(context, spark_df, "trx_enriched", data_environment=DataEnvironment.PRIVATE)

INFO: 2025-09-15 10:04:24 @ ### DataSet - writing started ###
INFO: 2025-09-15 10:04:37 @ ### DataSet - writing done, 600806 written, 0 corrupted, 0 rejected  ###


{'total_new_records': 600806,
 'corrupted_new_records': 0,
 'rejected_new_records': 0}

In [3]:
entity='customer'
cadence='monthly'

global_config = load_config('global.yaml', context=context)
global_config

{'bau': False,
 'monthly_features_look_back_in_months': 6,
 'monthly_data_horizon': None,
 'me_to_me_threshold': 1,
 'global_features_params': {'round_digits': 2},
 'tr_cfg_metadata': {'path': 'global.yaml', 'domain': 'demo_fuib'}}

In [4]:
config = load_config(f'{entity}/{cadence}/wrangling.yaml', context=context)
config

{'requested_features': {'z_score_sum_trx': {'v1': {'active': True,
    'train': True}},
  'z_score_cnt_trx': {'v1': {'active': True, 'train': True}},
  'z_score_sum_hghrsk_cntry': {'v1': {'active': True, 'train': True}},
  'cnt_trx_n_day': {'v1': {'active': True, 'train': True}},
  'sum_new_account': {'v1': {'active': True, 'train': True}},
  'max_trx': {'v1': {'active': True, 'train': True}},
  'sum_trx_cash': {'v1': {'active': True, 'train': True}},
  'sum_trx_cash_in': {'v1': {'active': True, 'train': True}},
  'sum_trx_cash_out': {'v1': {'active': True, 'train': True}},
  'cnt_trx_cash': {'v1': {'active': True, 'train': True}},
  'one_to_many': {'v1': {'active': True, 'train': False}},
  'many_to_one': {'v1': {'active': True, 'train': False}},
  'sum_pipe_customer': {'v1': {'active': True, 'train': False}},
  'sum_trx_fop': {'v1': {'active': True, 'train': False}},
  'sum_trx_name_mis': {'v1': {'active': True, 'train': False}},
  'cnt_trx_name_mis': {'v1': {'active': True, 'train':

In [5]:
features = features_discovery.get_features(config)
print(f'Calculating {len(features)} features')

Calculating 17 features


In [6]:
effective_config = global_config.copy()
effective_config.update(config)
# bau = effective_config.get('bau')
# if not bau:
#     train_config = load_config('build_train_dataset.yaml', context=context)
#     start_date = train_config.get('start_date')
#     end_date = train_config.get('end_date')
#     train_config
#     effective_config.update(train_config)
effective_config

{'bau': False,
 'monthly_features_look_back_in_months': 6,
 'monthly_data_horizon': None,
 'me_to_me_threshold': 1,
 'global_features_params': {'round_digits': 2},
 'tr_cfg_metadata': {'path': 'customer/monthly/wrangling.yaml',
  'domain': 'demo_fuib'},
 'requested_features': {'z_score_sum_trx': {'v1': {'active': True,
    'train': True}},
  'z_score_cnt_trx': {'v1': {'active': True, 'train': True}},
  'z_score_sum_hghrsk_cntry': {'v1': {'active': True, 'train': True}},
  'cnt_trx_n_day': {'v1': {'active': True, 'train': True}},
  'sum_new_account': {'v1': {'active': True, 'train': True}},
  'max_trx': {'v1': {'active': True, 'train': True}},
  'sum_trx_cash': {'v1': {'active': True, 'train': True}},
  'sum_trx_cash_in': {'v1': {'active': True, 'train': True}},
  'sum_trx_cash_out': {'v1': {'active': True, 'train': True}},
  'cnt_trx_cash': {'v1': {'active': True, 'train': True}},
  'one_to_many': {'v1': {'active': True, 'train': False}},
  'many_to_one': {'v1': {'active': True, 'train

In [ ]:
joined_trx_df: DataFrame = dataset_functions.read(context, 'trx_enriched', from_job_ts=Constants.BEGINNING_OF_TIME, data_environment=DataEnvironment.PRIVATE)
joined_trx_df.count()

In [7]:
joined_trx_df: DataFrame = dataset_functions.read(context, 'trx_enriched', data_environment=DataEnvironment.PUBLIC)
joined_trx_df.count()

601506

In [4]:
from pyspark.sql import functions as f
from pyspark.sql.types import StringType

# your inputs
target_customers = ["C_TUV2AWTLR5", "C_JDT6QDOETE", "C_62RFK5J4JG", "C_TD6RA5X3S", "C_4YC6D5T1WA", "C_KPPTFV6I92",
                   "C_STUK9D14IY", "C_A2QNYBS8HZ", "C_62RFK5J4JG", "C_VJ5OQP8S9A", "C_UTZ3LS1IRO", "C_JRA2MDILQD", "C_1CWRVD9G1M",
                   "C_ZJP8VM38XJ", "C_Q3QPU6S6D6", "C_STUK9D14IY"]

# share of CREDIT rows to make cash vs wire (0..1)
cash_share_credit = 0.60   # 60% -> cash_deposit, 40% -> wire_in
# share of DEBIT rows to make cash vs wire (0..1)
cash_share_debit  = 0.70   # 70% -> cash_withdrawal, 30% -> wire_out
rand_seed = 42             # change for a different mix but deterministic

start_june = f.lit("2025-06-01")
end_july   = f.lit("2025-07-01")

alias = f.concat(
    f.lit("DECL_"),
    f.upper(f.substring(f.sha2(f.coalesce(f.col("cp_id"), f.lit("NULL_CP")), 256), 1, 10))
)
alias2 = f.concat(
    f.lit("DECLX_"),
    f.upper(f.substring(f.sha2(f.concat(f.coalesce(f.col("cp_id"), f.lit("NULL_CP")), f.lit("_x")), 256), 1, 10))
)

is_target_row = (
    f.col("customer_id").isin(target_customers) &
    (f.col("txn_ts") >= start_june) &
    (f.col("txn_ts") < end_july)
)

new_declared = f.when(
    is_target_row,
    f.when(alias.eqNullSafe(f.col("counterparty_name")), alias2).otherwise(alias)
).otherwise(f.col("declared_counterparty_name"))

# --- mix cash & wire while matching direction ---
r = f.rand(seed=rand_seed)

# CREDIT rows: cash_deposit with probability cash_share_credit, else wire_in
credit_method = f.when(r < f.lit(cash_share_credit), f.lit("cash_deposit")) \
                 .otherwise(f.lit("wire_in"))

# DEBIT rows: cash_withdrawal with probability cash_share_debit, else wire_out
debit_method  = f.when(r < f.lit(cash_share_debit),  f.lit("cash_withdrawal")) \
                 .otherwise(f.lit("wire_out"))

new_method = (
    f.when(is_target_row & (f.col("direction") == f.lit("credit")), credit_method)
     .when(is_target_row & (f.col("direction") == f.lit("debit")),  debit_method)
     .otherwise(f.col("method"))   # keep original for other directions / out of scope
)

joined_trx_df = joined_trx_df.withColumn("declared_counterparty_name", new_declared)
joined_trx_df = joined_trx_df.withColumn("method", new_method)
joined_trx_df.count()

600806

In [5]:
from pyspark.sql import Row, functions as f
from datetime import datetime, timedelta
import random, string

def inject_cash_network_between_customers_mirrored(
    spark,
    df,                          # your DataFrame (e.g., joined_trx_df)
    customer_ids,                # list[str]
    *,
    june_year=2025,
    txns_per_customer=50,
    credit_share=0.5,
    credit_amount_usd=(500, 2000),
    debit_amount_usd=(600, 2500),
    seed=7
):
    random.seed(seed)

    june_start = datetime(june_year, 6, 1, 0, 0, 0)
    june_end   = datetime(june_year, 6, 30, 23, 59, 59)

    def _rand_id(prefix, n=12):
        return f"{prefix}_" + "".join(random.choices(string.ascii_uppercase + string.digits, k=n))
    shared_ip = "IP_NET_" + "".join(random.choices(string.ascii_uppercase + string.digits, k=10))

    # pull a light snapshot for targets (adjust fields if your schema differs)
    snap = (
        df.where(f.col("customer_id").isin(customer_ids))
          .select("customer_id","account_id","customer_name",
                  "pep_indicator","industry_code","rep_id_hash","address_hash","website_hash")
          .dropDuplicates(["customer_id","account_id"])
          .collect()
    )
    snap_by_id = {r["customer_id"]: r.asDict(recursive=True) for r in snap}
    for cid in customer_ids:
        if cid not in snap_by_id:
            snap_by_id[cid] = {
                "customer_id": cid,
                "account_id": None,
                "customer_name": None,
                "pep_indicator": False,
                "industry_code": None,
                "rep_id_hash": None,
                "address_hash": None,
                "website_hash": None,
            }

    def _rand_dt():
        span = int((june_end - june_start).total_seconds())
        return june_start + timedelta(seconds=random.randint(0, span))

    def _pick_amount(rng):
        lo, hi = rng
        return float(round(random.uniform(lo, hi), 2))

    base_schema = df.schema
    base_cols = [fld.name for fld in base_schema]

    def _row_for(*, cust, peer, ts, direction, amount_usd):
        """Return a dict with ALL columns from base schema populated (unknowns as None)."""
        rec = {c: None for c in base_cols}  # fill all columns with None first

        # Required core fields (use correct Python types)
        rec["txn_id"] = _rand_id("T")
        rec["txn_ts"] = ts
        rec["customer_id"] = cust["customer_id"]
        rec["account_id"] = cust.get("account_id")
        # KYC-ish
        if "customer_type" in rec:           rec["customer_type"] = "individual"
        if "customer_name" in rec:           rec["customer_name"] = cust.get("customer_name")
        if "pep_indicator" in rec:           rec["pep_indicator"] = bool(cust.get("pep_indicator", False))
        if "industry_code" in rec:           rec["industry_code"] = cust.get("industry_code")
        if "account_open_months" in rec:     rec["account_open_months"] = int(random.randint(1, 4))
        if "whitelist_flag" in rec:          rec["whitelist_flag"] = False

        # Transaction
        rec["direction"] = direction
        if "method" in rec:
            rec["method"] = "cash_deposit" if direction == "credit" else "cash_withdrawal"
        if "amount" in rec:                  rec["amount"] = amount_usd
        if "currency" in rec:                rec["currency"] = "USD"
        if "amount_usd" in rec:              rec["amount_usd"] = amount_usd
        if "purpose_code" in rec:            rec["purpose_code"] = "cash_activity"
        if "cross_border_flag" in rec:       rec["cross_border_flag"] = False
        if "balance_after_txn" in rec:       rec["balance_after_txn"] = 0.0
        if "is_split_component" in rec:      rec["is_split_component"] = False

        # Counterparty as another customer (network edge)
        if "cp_id" in rec:                   rec["cp_id"] = f"CUST_{peer['customer_id']}"
        if "cp_type" in rec:                 rec["cp_type"] = "internal_customer"
        if "cp_is_bank_customer" in rec:     rec["cp_is_bank_customer"] = True
        if "counterparty_name" in rec:       rec["counterparty_name"] = peer.get("customer_name")

        # Shared endpoint/IP
        if "endpoint_type" in rec:           rec["endpoint_type"] = "online_banking"
        if "ip_hash" in rec:                 rec["ip_hash"] = shared_ip

        # Linkage hashes (if present)
        if "rep_id_hash" in rec:             rec["rep_id_hash"] = cust.get("rep_id_hash")
        if "address_hash" in rec:            rec["address_hash"] = cust.get("address_hash")
        if "website_hash" in rec:            rec["website_hash"] = cust.get("website_hash")

        # Annotations
        if "scenario_type" in rec:           rec["scenario_type"] = "injected_cash_network_june"
        if "scenario_role" in rec:           rec["scenario_role"] = "participant"
        if "anomaly_flag" in rec:            rec["anomaly_flag"] = True

        return rec

    rows = []
    for cid in customer_ids:
        cust = snap_by_id[cid]
        peers = [p for p in customer_ids if p != cid]
        if not peers:
            continue

        for _ in range(txns_per_customer):
            peer = snap_by_id[random.choice(peers)]
            ts = _rand_dt()
            # source txn
            src_is_credit = (random.random() < credit_share)
            src_dir = "credit" if src_is_credit else "debit"
            src_amt = _pick_amount(credit_amount_usd if src_is_credit else debit_amount_usd)

            rows.append(_row_for(cust=cust, peer=peer, ts=ts, direction=src_dir, amount_usd=src_amt))

            # mirror txn
            mir_dir = "debit" if src_is_credit else "credit"
            rows.append(_row_for(cust=peer, peer=cust, ts=ts, direction=mir_dir, amount_usd=src_amt))

    if not rows:
        return df

    # Create with the base schema (all columns covered; None ok for nullable)
    injected_df = spark.createDataFrame([Row(**r) for r in rows], schema=base_schema)

    # --- Force required columns/values on injected rows ---
    base_fields = {fld.name: fld.dataType for fld in base_schema}

    # Set year_month = 2025-06-01 00:00:00 if column exists (cast to table dtype)
    if "year_month" in base_fields:
        injected_df = injected_df.withColumn(
            "year_month",
            f.lit("2025-06-01 00:00:00").cast(base_fields["year_month"])
        )

    # Set month_offset = 665 if column exists (cast to table dtype)
    if "month_offset" in base_fields:
        injected_df = injected_df.withColumn(
            "month_offset",
            f.lit(665).cast(base_fields["month_offset"])
        )
    # Create with the base schema (lengths match; None is valid for nullable fields)
    injected_df = spark.createDataFrame([Row(**r) for r in rows], schema=base_schema)

    # Union and return
    return df.unionByName(injected_df)


In [6]:
# Example:
target_customers = ["C_VJ5OQP8S9A", "C_54Y93I7W0K", "C_PNYHHE3OLD", "C_UJ1O5BN47F", "C_S3D5KZE1RW"]

joined_trx_df = inject_cash_network_between_customers_mirrored(
    spark,
    joined_trx_df,
    target_customers,
    txns_per_customer=70,             # each source produces a mirror, so 80 rows per customer on avg
    credit_share=0.5,
    credit_amount_usd=(5000, 10000),
    debit_amount_usd=(5500, 12000),
    seed=13
)

joined_trx_df.count()

601506

In [70]:
from pyspark.sql import functions as f

target_customers = ["C_VJ5OQP8S9A", "C_54Y93I7W0K", "C_PNYHHE3OLD", "C_UJ1O5BN47F", "C_S3D5KZE1RW"]

scope = (
    (f.col("customer_id").isin(target_customers)) &
    (f.col("txn_ts") >= f.lit("2025-06-01")) & (f.col("txn_ts") < f.lit("2025-07-01"))
)

# 1) How many injected rows?
joined_trx_df.where(scope & (f.col("scenario_type")=="injected_cash_network_june")).count()

# 2) Per-customer injected counts (June only)
joined_trx_df.where(scope & (f.col("scenario_type")=="injected_cash_network_june")) \
    .groupBy("customer_id").count().orderBy("customer_id").show(truncate=False)

# 3) Sanity: were only target customers touched?
joined_trx_df.where((f.col("scenario_type")=="injected_cash_network_june")) \
    .agg(
        f.countDistinct("customer_id").alias("distinct_customers_injected"),
        f.sum(f.when(~f.col("customer_id").isin(target_customers), 1).otherwise(0)).alias("rows_outside_target")
    ).show()


+------------+-----+
|customer_id |count|
+------------+-----+
|C_54Y93I7W0K|134  |
|C_PNYHHE3OLD|144  |
|C_S3D5KZE1RW|139  |
|C_UJ1O5BN47F|135  |
|C_VJ5OQP8S9A|148  |
+------------+-----+

+---------------------------+-------------------+
|distinct_customers_injected|rows_outside_target|
+---------------------------+-------------------+
|                          5|                  0|
+---------------------------+-------------------+



In [7]:
joined_trx_df = joined_trx_df.withColumn('month_offset', f.when(f.col("scenario_type")=="injected_cash_network_june", f.lit(665)).otherwise(f.col('month_offset')))

In [8]:
joined_trx_df = joined_trx_df.withColumn('amount_usd', f.when(f.col("scenario_type")=="injected_cash_network_june", (f.col('amount_usd')*0.05)).otherwise(f.col('amount_usd')))

In [9]:
joined_trx_df = joined_trx_df.withColumn('ip_hash', f.when(f.col("scenario_type")=="injected_cash_network_june", f.lit('192.168.1.1')).otherwise(f.col('ip_hash')))
joined_trx_df = joined_trx_df.withColumn('counterparty_ip', f.when(f.col("scenario_type")=="injected_cash_network_june", f.lit('192.168.1.1')).otherwise(f.col('counterparty_ip')))

In [10]:
for id in target_customers:
    joined_trx_df = joined_trx_df.withColumn('cp_id', f.when(f.col("cp_id")==f"CUST_{id}", f.lit(id)).otherwise(f.col('cp_id')))

In [8]:
from common.libs import dates as dates_lib

joined_trx_df = dates_lib.month_offset_to_year_month_columns(joined_trx_df, 'month_offset', 'year_month')

In [9]:
joined_trx_df.count()

601506

In [13]:
from thetaray.common.data_environment import DataEnvironment
dataset_functions.write(context, joined_trx_df, "trx_enriched", data_environment=DataEnvironment.PUBLIC)

INFO: 2025-09-15 10:08:12 @ ### DataSet - writing started ###
INFO: 2025-09-15 10:08:29 @ ### DataSet - writing done, 601506 written, 0 corrupted, 0 rejected  ###


{'total_new_records': 601506,
 'corrupted_new_records': 0,
 'rejected_new_records': 0}

In [14]:
from thetaray.common.data_environment import DataEnvironment
dataset_functions.publish(context, "trx_enriched", data_environment=DataEnvironment.PUBLIC)

INFO: 2025-09-15 10:08:59 @ finished publishing records for dataset trx_enriched 


True

In [39]:
joined_trx_df.filter(f.col('customer_id')=='C_54Y93I7W0K').count()

634

In [15]:
joined_trx_df.filter(f.col('customer_id')=='CUST_C_54Y93I7W0K').count()

0

In [16]:
joined_trx_df.filter(f.col('cp_id')=='CUST_C_54Y93I7W0K').count()

0

# Feature Executor

In [10]:
# wrangling_execution_strategy: WranglingExecutionStrategy = get_wrangling_execution_strategy(basic_execution_config, effective_config ,features)
# joined_trx_df = wrangling_execution_strategy.enrich_df_pre_feature_engineering(joined_trx_df)

features_executor = FeaturesExecutor(joined_trx_df, features, effective_config)
aggs_df = features_executor.execute()
aggs_df.count()

18000

# Widgets

In [11]:
investigated_entity_col = "customer_id"
cadence_col = "month_offset"
counterparty_col = "cp_id"
trx_amount_col = "amount_usd"
trx_direction_col = "direction"
direction_value_out = "debit"
direction_value_in = "credit"

## Populational Widgets

In [12]:
population = aggs_df.groupBy(cadence_col).agg(

    # in_cash_value
    # f.round(f.mean(f.when(f.col('in_cash_value') > 0, f.col('in_cash_value'))), 2).alias('pop_avg_in_cash_value'),
    # f.countDistinct(f.when(f.col('in_cash_value') > 0, f.col('cust'))).alias('custs_with_in_cash_value'),

    # out_cash_value
    # f.round(f.mean(f.when(f.col('out_cash_value') > 0, f.col('out_cash_value'))), 2).alias('pop_avg_out_cash_value'),
    # f.countDistinct(f.when(f.col('out_cash_value') > 0, f.col('cust'))).alias('custs_with_out_cash_value'),

    # total_cash_value
    # f.round(f.mean(f.when(f.col('total_cash_value') > 0, f.col('total_cash_value'))), 2).alias('pop_avg_cash_value'),
    # f.countDistinct(f.when(f.col('total_cash_value') > 0, f.col('cust'))).alias('pop_dstnct_trx_cash'),

    # sum trx
    f.round(f.mean(f.when(f.col('sum_trx') > 0, f.col('sum_trx'))), 2).alias('pop_avg_sum_trx'),
    
    #cnt_trx
    f.round(f.mean(f.when(f.col('cnt_trx') > 0, f.col('cnt_trx'))), 2).alias('pop_avg_cnt_trx'),
    f.countDistinct(f.when(f.col('cnt_trx') > 0, f.col(investigated_entity_col))).alias('pop_dstnct_cust_trx'),

#    sum_trx_cash
    f.round(f.mean(f.when(f.col('sum_trx_cash') > 0, f.col('sum_trx_cash'))), 2).alias('pop_avg_sum_trx_cash'),
    
    # cnt_trx_cash
    f.round(f.mean(f.when(f.col('cnt_trx_cash') > 0, f.col('cnt_trx_cash'))), 2).alias('pop_avg_cnt_trx_cash'),
    f.countDistinct(f.when(f.col('cnt_trx_cash') > 0, f.col(investigated_entity_col))).alias('pop_dstnct_cust_trx_cash'),
    
    # cnt_trx_n_day
    f.round(f.mean(f.when(f.col('cnt_trx_n_day') > 0, f.col('cnt_trx_n_day'))), 2).alias('pop_avg_cnt_trx_n_day'),

    # sum_new_account
    f.round(f.mean(f.when(f.col('sum_new_account') > 0, f.col('sum_new_account'))), 2).alias('pop_avg_new_account'),
    f.countDistinct(f.when(f.col('sum_new_account') > 0, f.col(investigated_entity_col))).alias('pop_dstnct_cust_new_account'),

#     # in_value
#     f.round(f.mean(f.when(f.col('in_value') > 0, f.col('in_value'))), 2).alias('pop_avg_in_value'),
#     f.countDistinct(f.when(f.col('in_value') > 0, f.col('cust'))).alias('custs_with_in_value'),

#     # out_value
#     f.round(f.mean(f.when(f.col('out_value') > 0, f.col('out_value'))), 2).alias('pop_avg_out_value'),
#     f.countDistinct(f.when(f.col('out_value') > 0, f.col('cust'))).alias('custs_with_out_value'),

#     # one_to_many
    # f.round(f.mean(f.when(f.col('one_to_many') > 0, f.col('one_to_many'))), 2).alias('pop_avg_one_to_many'),



#     # many_to_one
    # f.round(f.mean(f.when(f.col('many_to_one') > 0, f.col('many_to_one'))), 2).alias('pop_avg_many_to_one'),
#     f.countDistinct(f.when(f.col('many_to_one') > 0, f.col('cust'))).alias('custs_with_many_to_one'),



#     # total_atm_value
    # f.round(f.mean(f.when(f.col('total_atm_value') > 0, f.col('total_atm_value'))), 2).alias('pop_avg_total_atm_value'),
    # f.countDistinct(f.when(f.col('total_atm_value') > 0, f.col('cust'))).alias('custs_with_total_atm_value'),



#     # total_value
    # f.round(f.mean(f.when(f.col('total_value') > 0, f.col('total_value'))), 2).alias('pop_avg_total_value'),
    # f.countDistinct(f.when(f.col('total_value') > 0, f.col('cust'))).alias('custs_with_total_value'),

    # f.round(f.mean(f.when(f.col('total_pos_value') > 0, f.col('total_pos_value'))), 2).alias('pop_avg_total_pos_value'),
    # f.countDistinct(f.when(f.col('total_pos_value') > 0, f.col('cust'))).alias('custs_with_total_pos_value'),

#     # total_volume
    # f.round(f.mean(f.when(f.col('total_volume') > 0, f.col('total_volume'))), 2).alias('pop_avg_total_volume'),
#     f.countDistinct(f.when(f.col('total_volume') > 0, f.col('cust'))).alias('custs_with_total_volume'),

    # total_rapid_movement_value
    # f.round(f.mean(f.when(f.col('total_rapid_movement_value') > 0, f.col('total_rapid_movement_value'))), 2).alias('pop_avg_rapid_movement'),

    # f.round(f.mean(f.when(f.col('total_onprobation_value') > 0, f.col('total_onprobation_value'))), 2).alias('pop_avg_onprobation'),
    # f.countDistinct(f.when(f.col('total_onprobation_value') > 0, f.col('cust'))).alias('custs_with_onprobation'),

    # f.round(f.mean(f.when(f.col('total_risky_customer_value') > 0, f.col('total_risky_customer_value'))), 2).alias('pop_avg_risky_customer'),
    # f.countDistinct(f.when(f.col('total_risky_customer_value') > 0, f.col('cust'))).alias('custs_with_risky_customer'),

#     # total_hghrsk_cntry_val
#     f.round(f.mean(f.when(f.col('total_hghrsk_cntry_val') > 0, f.col('total_hghrsk_cntry_val'))), 2).alias('pop_total_hghrsk_cntry_val'),
#     f.countDistinct(f.when(f.col('total_hghrsk_cntry_val') > 0, f.col('cust'))).alias('custs_with_hghrsk_cntry_val'),

#     # total_hghrsk_cntry_vol
#     f.round(f.mean(f.when(f.col('total_hghrsk_cntry_vol') > 0, f.col('total_hghrsk_cntry_vol'))), 2).alias('pop_total_hghrsk_cntry_vol'),
#     f.countDistinct(f.when(f.col('total_hghrsk_cntry_vol') > 0, f.col('cust'))).alias('custs_with_hghrsk_cntry_vol')
)

aggs_df = aggs_df.join(population, cadence_col, how= 'left')
aggs_df.count()

18000

## Categorical Widgets
### One to Many

In [13]:
feature_name = "one_to_many"
feature_name_explainability = feature_name + "_explainability"

# Step 1: Aggregate the count and sum first
ds_agg_one_to_many = (
    joined_trx_df.filter( (f.col(trx_direction_col)==direction_value_out) & (~f.col(counterparty_col).isNull()) )
    .groupby(investigated_entity_col, cadence_col, counterparty_col)
    .agg(
        f.count('*').alias('count'),
        f.sum(trx_amount_col).alias('sum')
    )
).withColumn('sum', f.round(f.col('sum'), 2))

ds_agg_one_to_many = ds_agg_one_to_many.withColumn('sum', f.round(f.col('sum'), 2))

# Step 2: Perform collect_list aggregation separately
one_to_many_explainability = (
    ds_agg_one_to_many
    .groupby(investigated_entity_col, cadence_col)
    .agg(
        f.collect_list(
            f.struct(
                f.col(counterparty_col).alias('cn'),
                f.col('count').alias('c'),  # Now referencing from ds_agg_cred
                f.col('sum').alias('s')
            )
        ).alias(feature_name_explainability)
    )
    .select(investigated_entity_col,
        cadence_col,
        f.to_json(
            f.create_map(
                f.lit('data'),
                f.col(feature_name_explainability)
            )
        ).alias(feature_name_explainability)
    )
)

aggs_df = aggs_df.join(one_to_many_explainability.select(investigated_entity_col, cadence_col, feature_name_explainability), [investigated_entity_col, cadence_col], 'left')
aggs_df.count()

18000

### One to Many Concentration

In [14]:
import pyspark.sql.functions as f
from pyspark.sql.types import ArrayType, StructType, StructField, StringType, DoubleType, IntegerType

# Schema for parsing one_to_many_explainability
schema = StructType([
    StructField("data", ArrayType(
        StructType([
            StructField("cn", StringType(), True),
            StructField("c", IntegerType(), True),
            StructField("s", DoubleType(), True)
        ])
    ))
])

# Parse the JSON string
aggs_df= aggs_df.withColumn(
    "parsed_data",
    f.from_json(f.col("one_to_many_explainability"), schema)
)

# Sort array by 's' descending
aggs_df = aggs_df.withColumn(
    "sorted_data",
    f.expr("""
        array_sort(parsed_data.data, 
            (left, right) -> case 
                when left.s < right.s then 1 
                when left.s > right.s then -1 
                else 0 
            end
        )
    """)
)

# Top counterparty as struct wrapped in {"data":[...]}
aggs_df = aggs_df.withColumn(
    "cp_concentration_explainability",
    f.to_json(
        f.struct(
            f.array(f.element_at(f.col("sorted_data"), 1)).alias("data")
        )
    )
)

# Extract the s value of the top counterparty
aggs_df = aggs_df.withColumn(
    "cp_concentration_s",
    f.element_at(f.col("sorted_data"), 1).getItem("s")
)

# Sum all 's' values
aggs_df = aggs_df.withColumn(
    "total_s",
    f.expr("aggregate(parsed_data.data, double(0), (acc, x) -> acc + x.s)")
)

# Compute concentration ratio
aggs_df = aggs_df.withColumn(
    "cp_concentration",
    f.col("cp_concentration_s") / f.col("total_s")
)

aggs_df = aggs_df.drop('parsed_data', 'sorted_data', 'total_s')
aggs_df.count()

18000

### Many to One

In [15]:
feature_name = "many_to_one"
feature_name_explainability = feature_name + "_explainability"

# Step 1: Aggregate the count and sum first
ds_agg_many_to_one = (
    joined_trx_df.filter( (f.col(trx_direction_col)==direction_value_in) & (~f.col(counterparty_col).isNull()) )
    .groupby(investigated_entity_col, cadence_col, counterparty_col)
    .agg(
        f.count('*').alias('count'),
        f.sum(trx_amount_col).alias('sum')
    )
).withColumn('sum', f.round(f.col('sum'), 2))


# Step 2: Perform collect_list aggregation separately
many_to_one_explainability = (
    ds_agg_many_to_one
    .groupby(investigated_entity_col, cadence_col)
    .agg(
        f.collect_list(
            f.struct(
                f.col(counterparty_col).alias('cn'),
                f.col('count').alias('c'),  # Now referencing from ds_agg_cred
                f.col('sum').alias('s')
            )
        ).alias(feature_name_explainability)
    )
    .select(investigated_entity_col,
        cadence_col,
        f.to_json(
            f.create_map(
                f.lit('data'),
                f.col(feature_name_explainability)
            )
        ).alias(feature_name_explainability)
    )
)

aggs_df = aggs_df.join(many_to_one_explainability.select(investigated_entity_col, cadence_col, feature_name_explainability), [investigated_entity_col, cadence_col], 'left')
aggs_df.count()

18000

### FOP

In [16]:
feature_name = "sum_trx_fop"
feature_name_explainability = feature_name + "_explainability"

# Step 1: Aggregate the count and sum first
ds_agg_one_to_many = (
    joined_trx_df.filter( ((f.col('ip_hash')==f.col('counterparty_ip')) & (f.col('customer_id')!=f.col('cp_id'))) )
    .groupby(investigated_entity_col, cadence_col, counterparty_col)
    .agg(
        f.count('*').alias('count'),
        f.sum(trx_amount_col).alias('sum')
    )
).withColumn('sum', f.round(f.col('sum'), 2))

ds_agg_one_to_many = ds_agg_one_to_many.withColumn('sum', f.round(f.col('sum'), 2))

# Step 2: Perform collect_list aggregation separately
one_to_many_explainability = (
    ds_agg_one_to_many
    .groupby(investigated_entity_col, cadence_col)
    .agg(
        f.collect_list(
            f.struct(
                f.col(counterparty_col).alias('cn'),
                f.col('count').alias('c'),  # Now referencing from ds_agg_cred
                f.col('sum').alias('s')
            )
        ).alias(feature_name_explainability)
    )
    .select(investigated_entity_col,
        cadence_col,
        f.to_json(
            f.create_map(
                f.lit('data'),
                f.col(feature_name_explainability)
            )
        ).alias(feature_name_explainability)
    )
)

aggs_df = aggs_df.join(one_to_many_explainability.select(investigated_entity_col, cadence_col, feature_name_explainability), [investigated_entity_col, cadence_col], 'left')
aggs_df.count()

18000

### High Risk Country

In [17]:
# Step 1: Aggregate the count and sum first
ds_agg_high_risk_country = (
    joined_trx_df.filter(f.col("cp_country_risk_level").isin(['High','Medium','Low']))
    .groupby(investigated_entity_col, cadence_col, 'cp_jurisdiction')
    .agg(
        f.count('*').alias('count'),
        f.sum(trx_amount_col).alias('sum'),
        f.max('cp_country_risk_level').alias("cp_country_risk_level")
    )
).withColumn('sum', f.round(f.col('sum'), 2))


# Step 2: Perform collect_list aggregation separately
high_risk_country_explainability = (
    ds_agg_high_risk_country
    .groupby(investigated_entity_col, cadence_col)
    .agg(
        f.collect_list(
            f.struct(
                f.col('cp_jurisdiction').alias('ct'),
                f.col('cp_country_risk_level').alias('cr'),
                f.col('count').alias('c'),  # Now referencing from ds_agg_cred
                f.col('sum').alias('s')
            )
        ).alias('high_risk_country_explainability')
    )
    .select(investigated_entity_col,
        cadence_col,
        f.to_json(
            f.create_map(
                f.lit('data'),
                f.col('high_risk_country_explainability')
            )
        ).alias('high_risk_country_explainability')
    )
)

aggs_df = aggs_df.join(high_risk_country_explainability.select(investigated_entity_col, cadence_col,'high_risk_country_explainability'), [investigated_entity_col, cadence_col], 'left')
aggs_df.count()

18000

In [18]:
from common.libs import dates as dates_lib

aggs_df = dates_lib.month_offset_to_year_month_columns(aggs_df, 'month_offset', 'year_month')

In [19]:
aggs_df = aggs_df.drop('sum_in_out_ratio')

In [20]:
aggs_df = aggs_df.fillna(0)

In [21]:
dataset_functions.write(context, aggs_df, 'customer_monthly', data_environment=DataEnvironment.PUBLIC)

INFO: 2025-09-21 15:44:44 @ ### DataSet - writing started ###
INFO: 2025-09-21 15:45:27 @ ### DataSet - writing done, 18000 written, 0 corrupted, 0 rejected  ###


{'total_new_records': 18000,
 'corrupted_new_records': 0,
 'rejected_new_records': 0}

In [22]:
from thetaray.common.data_environment import DataEnvironment
dataset_functions.publish(context, 'customer_monthly', data_environment=DataEnvironment.PUBLIC)

INFO: 2025-09-21 15:45:30 @ finished publishing records for dataset customer_monthly 


True

In [59]:
context.close()